In [1]:
import requests
import json
import arrow
import os
import hashlib
import sys
from ics import Calendar, Event
from ics.contentline.container import Container, ContentLine
from ics.types import ContainerItem
from datetime import datetime

In [2]:
# Static Values
headers = {
    'x-api-key': "0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z" # public api key! This key does not have to be kept private
}
output_dir = "./leagues/"

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)


In [3]:
def generate_empty_calender(name):
    c = Calendar(creator="ics.tools Combinder v1.0")
    c.extra.append(ContentLine(name='NAME', value=name))
    c.extra.append(ContentLine(name='X-WR-CALNAME', value=name))
    c.extra.append(ContentLine(name='METHOD', value="PUBLISH"))
    return c

In [4]:
def get_leagues():
    r=requests.get("https://esports-api.lolesports.com/persisted/gw/getLeagues", headers=headers, params={"hl": "de-DE"})
    if r.status_code == 200:
        return json.loads(r.content)['data']['leagues']
    else:
        print("No leagues available")
        print(r)
        sys.exit(1)

In [5]:
def get_schedule(league_ids, pageToken):
    r=requests.get("https://esports-api.lolesports.com/persisted/gw/getSchedule", headers=headers, params={
        "hl": "de-DE",
        "leagueId": ",".join(league_ids),
        "pageToken": pageToken
    })
    if r.status_code == 200:
        content = json.loads(r.content)['data']['schedule']
        return content['pages']['older'], content['events']
    else:
        print("No schedules available")
        print(r)
        sys.exit(1)

In [6]:
def get_schedule_all_pages(league_ids):
    events = []
    token = None
    while True:
        token, evts = get_schedule(league_ids, token)
        print("Got page with {} events".format(len(evts)))
        if evts == None:
            break
        for e in evts:
            events.append(e)
        if token == None:
            break
    return events

In [7]:
def get_description(l_event):
    #Liga: WM
    #Phase: Gruppenphase
    #Teams: Blub vs. Nano
    #Strategy: bestOf 5
    #Status: 
    #Ergebnisse: Blub (5) : Nano (1)
    desc = f"Liga: {l_event['league']['name']}\n"
    desc += f"Phase: {l_event['blockName']}\n"
    desc += f"Teams: {l_event['match']['teams'][0]['name']} vs. {l_event['match']['teams'][1]['name']}\n"
    desc += f"Spiel-Strategie: {l_event['match']['strategy']['type']} {l_event['match']['strategy']['count']}\n"
    desc += f"Status: {l_event['state']}\n"
    if l_event['state'] == "completed":
        desc += f"Ergebnisse: {l_event['match']['teams'][0]['name']} ({l_event['match']['teams'][0]['result']['gameWins']}) vs. {l_event['match']['teams'][1]['name']} ({l_event['match']['teams'][1]['result']['gameWins']})\n"
    return desc

In [8]:
leagues = get_leagues()
league_ids = list(map(lambda d: d['id'], leagues))
league_calendar = {}

for league in leagues:
    if os.path.isfile(output_dir + league['slug'] + ".ics"):
        with open(output_dir + league['slug'] + ".ics", "r") as file1:
            l = file1.read()
        league_calendar[league['slug']] = Calendar(l)
        print("Old League found", league['slug'])
    else:
        league_calendar[league['slug']] = generate_empty_calender(f"{league['name']} Spielplan")
        print("New League found", league['slug'])

Ol League found european-masters
Ol League found turkey-academy-league
Ol League found lla
Ol League found pcs
Ol League found superliga
Ol League found ultraliga
Ol League found primeleague
Ol League found pg_nationals
Ol League found nlc
Ol League found liga_portuguesa
Ol League found lfl
Ol League found hitpoint_masters
Ol League found greek_legends
Ol League found esports_balkan_league
Ol League found dutch_league
Ol League found belgian_league
Ol League found baltic_masters
Ol League found cblol_academy
Ol League found lco
Ol League found ljl_academy
Ol League found vcs
Ol League found worlds
Ol League found all-star
Ol League found lcs
Ol League found lec
Ol League found lck
Ol League found lpl
Ol League found msi
Ol League found oce-opl
Ol League found cblol-brazil
Ol League found lck_challengers_league
Ol League found turkiye-sampiyonluk-ligi
Ol League found ljl-japan
Ol League found lcl
Ol League found lcs-academy


In [9]:
events = get_schedule_all_pages(league_ids)
events.sort(key=lambda e: e['match']['id'])

Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80

In [10]:
len(events)

9572

In [11]:
for l_event in events:
    # Skip current running games
    if l_event['state'] == "inProgress" or not "match" in l_event:
        continue
    calendar = league_calendar[l_event['league']['slug']]
    uid = l_event['league']['slug'] + "-" + l_event['match']['id'] + "@lol.ics.tools"
    event = None
    is_new = True
    # Suche nach Kalender Event für Match Id
    for tevent in calendar.events:
        if tevent.uid == uid:
            event = tevent
            is_new = False
            break
    
    if is_new:
        print("New event for " + uid)
        event = Event()
        event.uid = uid
        event.created = datetime.now()
        
    event_name = f"[{l_event['league']['name']}] {l_event['match']['teams'][0]['code']} vs. {l_event['match']['teams'][1]['code']}"
    event_begin = arrow.get(l_event['startTime'])
    event_end = event_begin.shift(hours=l_event['match']['strategy']['count'])
    event_description = get_description(l_event)
    changed = False
    if event_name != event.summary:
        changed = True
        print("Name changed")
        event.summary = event_name
    
    if event_begin != event.begin:
        changed = True
        print("Start changed")
        event.begin = event_begin.datetime
    
    if event_end != event.end:
        changed = True
        print("End changed")
        event.end = event_end.datetime
    
    if event_description != event.description:
        changed = True
        print("Description changed")
        event.description = event_description
    
    if changed:
        event.last_modified = arrow.now().datetime
        
    if is_new:
        calendar.events.append(event)

In [12]:
sum = 0
for key in league_calendar:
    sum += len(league_calendar[key].events)
print("Found", sum, "Events in Calendars")


Found 9572 Events in Calendars


In [13]:
# Save all calendars in individual files
for key in league_calendar:
    with open(output_dir + "/" + key + ".ics", 'w') as file:
        file.writelines(league_calendar[key])

../leagues//european-masters.ics
../leagues//turkey-academy-league.ics
../leagues//lla.ics
../leagues//pcs.ics
../leagues//superliga.ics
../leagues//ultraliga.ics
../leagues//primeleague.ics
../leagues//pg_nationals.ics
../leagues//nlc.ics
../leagues//liga_portuguesa.ics
../leagues//lfl.ics
../leagues//hitpoint_masters.ics
../leagues//greek_legends.ics
../leagues//esports_balkan_league.ics
../leagues//dutch_league.ics
../leagues//belgian_league.ics
../leagues//baltic_masters.ics
../leagues//cblol_academy.ics
../leagues//lco.ics
../leagues//ljl_academy.ics
../leagues//vcs.ics
../leagues//worlds.ics
../leagues//all-star.ics
../leagues//lcs.ics
../leagues//lec.ics
../leagues//lck.ics
../leagues//lpl.ics
../leagues//msi.ics
../leagues//oce-opl.ics
../leagues//cblol-brazil.ics
../leagues//lck_challengers_league.ics
../leagues//turkiye-sampiyonluk-ligi.ics
../leagues//ljl-japan.ics
../leagues//lcl.ics
../leagues//lcs-academy.ics


In [17]:
# write a ics file all events
c = generate_empty_calender("League of Legends Profispiele")
for key in league_calendar:
    for event in league_calendar[key].events:
        c.events.append(event)
with open(output_dir + "/" + "alle.ics", 'w') as file:
    file.writelines(c)